In [1]:
import sys
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PYSPARK_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 

In [2]:
from pyspark.sql import SparkSession

sc = SparkSession.Builder().appName('vif').getOrCreate()

22/12/23 14:17:58 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.213 instead (on interface wlp5s0)
22/12/23 14:17:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/23 14:17:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [163]:
import pandas as pd
import numpy as np


N = 5000
data = pd.DataFrame({
    'num1': np.random.normal(1000, 100, size=N), 
    'num2': np.random.randint(low=-10, high=10, size=N),
    'num3': np.random.randint(low=-10, high=10, size=N),
    'num4': np.random.randint(low=-10, high=10, size=N),
    'num5': np.random.randint(low=-10, high=10, size=N),
    'target': np.random.choice([0, 1],p=[0.9, 0.1],  size=N)
})

data = data.assign(num3=data.num2 * 3)

dd = sc.createDataFrame(data)
dd.show()

+------------------+----+----+----+----+------+
|              num1|num2|num3|num4|num5|target|
+------------------+----+----+----+----+------+
| 984.8872605711517|   7|  21|  -8|  -7|     0|
|1099.2159034072686|  -4| -12|   6|  -9|     0|
|1015.1058028043047|  -3|  -9|   4|  -4|     0|
| 985.6084462370345| -10| -30|  -9| -10|     0|
|1100.3370751658183|  -9| -27|  -6| -10|     0|
| 1063.782948966551| -10| -30|   6|   9|     0|
|1140.6366426432483| -10| -30|  -6|   3|     0|
|1106.4655584201712|   7|  21|  -1|  -3|     0|
|1109.0426584457339|   2|   6|  -1|  -4|     0|
|1067.7017606269155|  -5| -15|   8|  -3|     0|
|1065.7883926650575|   1|   3|   0|   2|     0|
|1141.1533747176907|   2|   6|   7|  -3|     0|
|1054.1722186621575|  -9| -27|  -3|   3|     0|
|1052.4024936957942|  -3|  -9|   6|  -6|     0|
| 966.2962217837946| -10| -30|  -5|   9|     0|
| 980.4324534504395|   5|  15|  -4|  -8|     0|
|1043.0995064195083|   8|  24|  -5|   2|     0|
| 933.0004552758113|  -8| -24|   9|  -5|

/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [164]:
from pyspark.sql.types import Row

# Importing required libraries for VIF Calculation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

In [ ]:
vif_threshold = 5 #Threshold for VIF

class VifFilter:
  def __init__(self,
    label_column,
    exclude_cols=None,
    vif_threshold=5,
    estimator=LogisticRegression,
    evaluator=BinaryClassificationEvaluator
  ):

    self.label_column = label_column
    self.exclude_cols = exclude_cols
    self.vif_threshold = vif_threshold
    self.estimator = estimator
    self.evaluator = evaluator

  def _reorder_label(self, data):
    '''Ensures label is LAST column'''
    columns = data.columns
    columns.remove(self.label_column)
    columns.append(self.label_column)
    return data.select(*[columns])


  def calc_vif(self, data, xvar_names, vif_max, colnum_max):
    print(f'Number columns at this level: {len(data.columns)}')
    vif_max = self.vif_threshold
    for i in range(2, len(xvar_names)):
      train_table = data.rdd.map(lambda x: [Vectors.dense(x[2:i]+x[i+1:]), x[i]]).toDF(['features', 'label'])
      lr = self.estimator(featuresCol='features', labelCol='label')
      lr_model = lr.fit(train_table)
      predictions = lr_model.transform(train_table)
      evaluator = self.evaluator(predictionCol='prediction', labelCol='label')
      r_sq = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
      vif = 1/(1-r_sq)
      if vif_max < vif:
        vif_max = vif
        colnum_max = i

      return vif_max, colnum_max

  def remove_collinear(self, data):

    data = self._reorder_label(data)

    xvar_names = data.columns
    colnum_max = len(xvar_names)
    vif_max = self.vif_threshold + 1

    while vif_max > 5:
      vif_max, colnum_max = self.calc_vif(data, xvar_names, vif_max, colnum_max)
      if vif_max > 5:
        data = data.drop(data[colnum_max])
        xvar_names = data.columns
    else:
      return data

def vif_cal_iter(inputdata,vif_threshold):
  xvar_names = inputdata.columns
  global vif_max
  global colnum_max
  colnum_max = 10000 # Initialising with a fake value
  vif_max = vif_threshold + 1
  def vif_cal(inputdata, xvar_names, vif_max, colnum_max, vif_threshold):
    print("Dimension of table at this level")
    print("================================")
    print(inputdata.count(), len(inputdata.columns))
    print("List of X Variables")
    print("===================")
    print(xvar_names)
    vif_max = vif_threshold
    for i in range(2,len(xvar_names)):
      train_t = inputdata.rdd.map(lambda x: [Vectors.dense(x[2:i]+x[i+1:]), x[i]]).toDF(['features', 'label'])
      lr = LinearRegression(featuresCol = 'features', labelCol = 'label')
      lr_model = lr.fit(train_t)
      predictions = lr_model.transform(train_t)
      evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label')
      r_sq=evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
      vif=1/(1-r_sq)
      if vif_max < vif:
        vif_max = vif
        colnum_max = i
    return vif_max, colnum_max
  while vif_max > 5:
    vif_max, colnum_max = vif_cal(inputdata, xvar_names, vif_max, colnum_max, vif_threshold)
    if vif_max > vif_threshold:
        print("Start of If Block")
        inputdata = inputdata.drop(inputdata[colnum_max])
        xvar_names = inputdata.columns
        print("Dimension of table after this iteration")
        print("=======================================")
        print(inputdata.count(), len(inputdata.columns))
        print("List of X Variables remaining")
        print("=============================")
        print(xvar_names)
  else:
    return inputdata

In [36]:
data = dd
col_names = data.columns
col_names.remove('target') # works inplace
col_names

['num1', 'num2', 'num3', 'num4', 'num5']

In [ ]:
# drop actual target?

In [37]:
data.head()

Row(num1=936.4082310320396, num2=5, num3=15, num4=2, num5=-6, target=0)

In [184]:
class VifFilter: pass

def calc_vif(self, data, target_name=None):
    if target_name and target_name in data.columns:
        data = data.drop(target_name)

    vifs = pd.Series(name='vif', dtype='float64')
    col_names = list(data.columns)
    for idx, colname in enumerate(col_names):
        train_table = data.rdd.map(lambda x: [Vectors.dense(x[:idx]+x[idx+1:]), x[idx]]).toDF(['features', 'label'])
        lr = LinearRegression(featuresCol='features', labelCol='label') 
        lr_model = lr.fit(train_table)
        predictions = lr_model.transform(train_table)
        eval = RegressionEvaluator(predictionCol='prediction', labelCol='label') 
        r_sq = eval.evaluate(predictions, {eval.metricName: 'r2'})

        try:
            vif = 1/(1-r_sq)
        except ZeroDivisionError as z:
            # this means perfect correlation, set to high number
            vif = float('Inf')
        vifs[colname] = vif

    return vifs.sort_values()
VifFilter.calc_vif = calc_vif

In [185]:
vf = VifFilter()
vf.calc_vif(dd, target_name='target')

22/12/23 15:54:32 WARN Instrumentation: [90cfa142] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:54:32 WARN Instrumentation: [90cfa142] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 15:54:33 WARN Instrumentation: [9d8aea5d] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:54:34 WARN Instrumentation: [9e3fc421] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:54:34 WARN Instrumentation: [392af55b] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:54:35 WARN Instrumentation: [392af55b] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 15:54:35 WARN Instrumentation: [b6fbc8ee] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:54:35 WARN Instrumentation: [b6fbc8ee] Cholesky solver failed due to s

num4    1.000641
num1    1.000703
num5    1.000932
num2         inf
num3         inf
Name: vif, dtype: float64

In [206]:
from pyspark.ml.stat import ChiSquareTest

def calc_target_corr(self, data, target_name):
    if target_name not in data.columns:
        raise AttributeError(f'{target_name} not found in data')

    # reorganize column order
    col_names = list(data.columns)
    col_names.remove(target_name)
    col_names.append(target_name)

    data = data.select(*[col_names])
    train_table = data.rdd.map(lambda x: [Vectors.dense(x[:-1]), x[-1]]).toDF(['features', 'label'])
    chi2res = ChiSquareTest.test(train_table, 'features', 'label', flatten=True).toPandas()
    chi2res = chi2res.assign(featureName=col_names[:-1])[['featureName', 'pValue', 'statistic']]

    return chi2res.set_index('featureName')


VifFilter.calc_chi2_corr = calc_target_corr
vf = VifFilter()

In [194]:
chi2_corr= vf.calc_chi2_corr(dd, target_name='target')
vifs = vf.calc_vif(data, target_name='target')

22/12/23 15:55:39 WARN Instrumentation: [18fa28a7] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:55:39 WARN Instrumentation: [18fa28a7] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 15:55:39 WARN Instrumentation: [63fb0fb4] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:55:40 WARN Instrumentation: [cdf47fea] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:55:41 WARN Instrumentation: [37d86159] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:55:41 WARN Instrumentation: [37d86159] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 15:55:42 WARN Instrumentation: [5a46919b] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 15:55:42 WARN Instrumentation: [5a46919b] Cholesky solver failed due to s

In [158]:
dd.show()

+------------------+----+----+----+----+------+
|              num1|num2|num3|num4|num5|target|
+------------------+----+----+----+----+------+
| 983.4712302534647|   1|   3|   2|   1|     0|
| 979.0702829763949|   5|  15|   2|   4|     0|
|1055.8307118259404| -10| -30|  -9|  -1|     0|
| 1037.987618082322|  -8| -24|  -9|  -8|     1|
|1169.7448679455354|  -5| -15|  -3|   9|     0|
+------------------+----+----+----+----+------+



In [207]:
vf.data = dd 
vf.vif_threshold = 5
vf.target_name = 'target'

In [208]:
self = vf
self.vif_threshold

5

In [209]:
max_vif = float('Inf')
col_names = self.data.columns
chi2_corr = self.calc_chi2_corr(self.data, self.target_name)


data = dd #self.data

while max_vif > self.vif_threshold:
    vifs = self.calc_vif(data, target_name=self.target_name)
    vif_chi2 = (
        vifs.to_frame()
        .merge(chi2_corr, left_index=True, right_index=True)
        .sort_values(['vif', 'statistic'], ascending=True)
    )

    # drop highest column value
    col_names_reduced = vif_chi2.index[:-1]
    print(col_names)
    break

22/12/23 16:02:37 WARN Instrumentation: [6568e499] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:38 WARN Instrumentation: [6568e499] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 16:02:38 WARN Instrumentation: [887d8047] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:39 WARN Instrumentation: [430a21e0] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:40 WARN Instrumentation: [3b60b1c4] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:40 WARN Instrumentation: [3b60b1c4] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 16:02:40 WARN Instrumentation: [d188a241] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:41 WARN Instrumentation: [d188a241] Cholesky solver failed due to s

In [228]:
class VIFFilter:
    def __init__(self, 
        vif_threshold,
        target_name,
    ) -> None:
        self.vif_threshold = vif_threshold
        self.target_name = target_name


    def remove_mulitocollinear(self, data, max_iter=None):
        max_vif = float('Inf')
        col_names = data.columns
        chi2_corr = self.calc_target_corr(data, self.target_name)
        it = 0
        while max_vif > self.vif_threshold:
            if max_iter and it > max_iter:
                break
            vifs = self.calc_vif(data, target_name=self.target_name)
            vif_chi2 = (
                vifs.to_frame()
                    .merge(chi2_corr, left_index=True, right_index=True)
                    .sort_values(['vif', 'statistic'], ascending=True)
            )

            col_names_reduced = vif_chi2.index[:-1]
            print(col_names_reduced)
            data = data.select(*col_names_reduced)

            max_vif = vifs.max()

        return data

    
    def calc_target_corr(self, data, target_name):
        if target_name not in data.columns:
            raise AttributeError(f'{target_name} not found in data')

        # reorganize column order
        col_names = list(data.columns)
        col_names.remove(target_name)
        col_names.append(target_name)

        data = data.select(*[col_names])
        train_table = data.rdd.map(lambda x: [Vectors.dense(x[:-1]), x[-1]]).toDF(['features', 'label'])
        chi2res = ChiSquareTest.test(train_table, 'features', 'label', flatten=True).toPandas()
        chi2res = chi2res.assign(featureName=col_names[:-1])[['featureName', 'pValue', 'statistic']]

        return chi2res.set_index('featureName')

    def calc_vif(self, data, target_name=None):
        if target_name and target_name in data.columns:
            data = data.drop(target_name)

        vifs = pd.Series(name='vif', dtype='float64')
        col_names = list(data.columns)
        for idx, colname in enumerate(col_names):
            train_table = data.rdd.map(lambda x: [Vectors.dense(x[:idx]+x[idx+1:]), x[idx]]).toDF(['features', 'label'])
            lr = LinearRegression(featuresCol='features', labelCol='label', regParam=0.5) 
            lr_model = lr.fit(train_table)
            predictions = lr_model.transform(train_table)
            eval = RegressionEvaluator(predictionCol='prediction', labelCol='label') 
            r_sq = eval.evaluate(predictions, {eval.metricName: 'r2'})

            try:
                vif = 1/(1-r_sq)
            except ZeroDivisionError as z:
                # this means perfect correlation, set to high number
                vif = float('Inf')
            vifs[colname] = vif

        return vifs.sort_values()

In [229]:
vf = VIFFilter(
    5, 
    'target'
)

In [230]:
vf.remove_mulitocollinear(dd, max_iter=2)

Index(['num4', 'num1', 'num5', 'num2'], dtype='object')
Index(['num2', 'num4', 'num1'], dtype='object')


DataFrame[num2: bigint, num4: bigint, num1: double]

In [222]:
vifs.max()

inf